# [![](https://img.shields.io/badge/Video-Hướng%20dẫn-ff0000)](https://youtu.be/798qchqWty4) [![](https://img.shields.io/badge/Design-stablediffusion.vn-0075ff)](https://stablediffusion.vn) [![](https://img.shields.io/badge/Ver-2.0-0075ff)](https://stablediffusion.vn/update/) [![](https://img.shields.io/badge/All%20Tools-sdvn.me-0075ff)](https://stablediffusion.vn/bo-cong-cu/) [![](https://img.shields.io/badge/SDVN-Library-green)](https://bit.ly/sdvn-lib) [![](https://img.shields.io/badge/Khoá%20học-SD%20Flux-red)](https://hungdiffusion.com/) [![](https://img.shields.io/badge/Group-Support-0075ff)](https://www.facebook.com/groups/stablediffusion.vn) [![](https://img.shields.io/discord/813085864355037235?color=blue&label=Discord&logo=Discord)](https://discord.gg/5SEtApPeyG)
---
🧨 `Train được ở L4 (Yêu cầu bật low VRAM - Batch_size 1) và A100`

🔥 `Cập nhật phiên bản v2, hỗ trợ train nhiều lora trong một setting, thêm chế độ sử dụng với giao diện UI`

#☕️ 1.Cài đặt

In [ ]:
#@title ✨ Cài đặt UI

# @markdown 🟡 `Lựa chọn Gradio nếu muốn train trực tiếp với giao diện UI (Sử dụng link gradio) `
%cd /content
from google.colab import drive
drive.mount('/content/drive')

UI = 'Colab' # @param ['Gradio','Colab']

root_dir = "/content"
repo_dir = f"{root_dir}/SDVN-training-colab-flux"
toolkit_dir = f"{repo_dir}/ai-toolkit"
config_dir = f"{repo_dir}/Config"

%cd {root_dir}
import os
if not os.path.exists(repo_dir):
  !git clone  --recurse-submodules https://github.com/StableDiffusionVN/SDVN-training-colab-flux {repo_dir}
  %env TF_CPP_MIN_LOG_LEVEL=1
  %env LD_PRELOAD={repo_dir}/libtcmalloc_minimal.so.4
%cd {toolkit_dir}
!pip install -q -r requirements.txt

%run {repo_dir}/TrainScript.ipynb

if UI == "Gradio":
  !rm -r {toolkit_dir}/output
  !ln -s /content/drive/MyDrive/SD-Data/Lora {toolkit_dir}/output
  !python {toolkit_dir}/flux_train_ui.py


#⏳ 2. Cấu hình train

In [ ]:
#@title ✨ 2.1 Xử lý dữ liệu
# @markdown 🟡 `Train nhiều lora bằng cách nhập nhiều thư mục, cách nhau bằng dấu phẩy (,)`
TrainFolder = "/content/drive/MyDrive/SD-Data/TrainData"  # @param {type:'string'}
DataClean = False #@param {type:"boolean"}
# @markdown 🟡 `Điều chỉnh mức độ dài caption tự động`
Caption = 'Florence' # @param ['None','Blip','Florence']
Caption_Length = "Medium" # @param ['Short','Medium','Long']
# @markdown 🟡 `Thêm caption tuỳ chọn`
Custom_Caption = "" # @param {type:'string'}
AddFolderName = False #@param {type:"boolean"}
Remove_Caption = False #@param {type:"boolean"}
Append = False #@param {type:"boolean"}
extension =".txt"
train_folder_list = TrainFolder.split(",")

Cap_prompt = {
    'Short':['<CAPTION>',10,30],
    'Medium':['<DETAILED_CAPTION>',10,100],
    'Long':['<MORE_DETAILED_CAPTION>',10,150]
}

for folder in train_folder_list:
  if DataClean == True :
    %cd /content
    clean_directory(folder)
  if Caption != "None":
    caption_dir(folder,Cap_prompt[Caption_Length][0])
  if AddFolderName:
    add_forder_name(folder)
  if Custom_Caption != "":
    process_dir(folder, Custom_Caption, Append, Remove_Caption)

In [ ]:
#@title ⚙️ 2.2 Cài đặt train
Low_VRAM = True #@param {type:'boolean'}
Lora_name = "lora_name" # @param {"type":"string","placeholder":"Tên lora"}
OutputFolder = "/content/drive/MyDrive/SD-Data/Lora" #@param {type:'string'}
# @markdown 🟡 `Điều chỉnh steps theo mức độ train mong muốn, các thư mục đặt tên theo cú pháp {steps}_{name} sẽ tự nhận steps riêng theo tên`
Steps = 1000 #@param {type:'number'}
Save_steps = 250 #@param {type:'number'}
Batch_size = 4 #@param {type:'number'}
# @markdown 🟡 `Các thông số nên để theo mặc định`
Lr = 1e-4 #@param {type:'number'}
Dim = 16 #@param {type:'number'}
Alpha = 16 #@param {type:'number'}
Resolution = "[ 512, 768, 1024 ]" #@param {type:'string'}


folder_train = []
for folder in train_folder_list:
  if len(get_supported_images(folder)) > 0:
    folder_dic = {
    "path": folder,
    "steps": get_steps(folder)[0],
    "name": get_steps(folder)[1].replace(" ", "_")
    }
    folder_train.append(folder_dic)

for folder in folder_train:
  file_path = f'{config_dir}/config_{folder["name"]}.yaml'
  !cp "{config_dir}/config.yaml" "{file_path}"
  if AddFolderName:
    process_dir(folder["path"], folder["name"], False, False)
  config = {
      'data': [r'folder_path:.*',f'folder_path: "{folder["path"]}"'],
      'name': [r'name:.*',f'name: "{Lora_name if len(folder_train) == 1 else folder["name"]}"'],
      'output': [r'training_folder:.*',f'training_folder: "{OutputFolder}"'],
      'steps': [r' steps:.*',f' steps: {folder["steps"]}'],
      'save_steps': [r'save_every:.*',f'save_every: {Save_steps}'],
      'sample_every': [r'sample_every:.*',f'sample_every: {Save_steps}'],
      'resolution': [r'resolution:.*',f'resolution: {Resolution}'],
      'batch': [r'batch_size:.*',f'batch_size: {Batch_size}'],
      'dim': [r'linear:.*',f'linear: {Dim}'],
      'alpha': [r'linear_alpha:.*',f'linear_alpha: {Alpha}'],
      'lr': [r'lr:.*',f'lr: {Lr}'],
      'prompt': [r'prompts:[\s\S]*?neg:',f'prompts:\n          - "{random_sample(folder["path"])}"\n        neg:'],
      'low_vram': [r'low_vram:.*',f'low_vram: {"true" if Low_VRAM == True else "false"}'],

  }
  for i in config:
    replace(config[i][0],config[i][1])

  print('=====================')
  print(f'Thư mục train: {folder["path"]}')
  print(f'  Số lượng ảnh: {len(get_supported_images(folder["path"]))}')
  print(f'  Số Steps: {folder["steps"]}')
  print(f'  Output: {OutputFolder}/{folder["name"]}')
  print('=====================')


#3. 🧨 Train

In [ ]:
#@title ⏳ Run Lora Train
train()